# Packages

In [1]:
import os
                
import scipy
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from bioat.lib.libpandas import set_option
from bioat.lib.libpath import HOME
from pybedtools import BedTool
set_option()

INFO  @ 2023-08-28 22:00:02 bioat.lib.libpandas.set_option: set pandas: max_colwidth=40
INFO  @ 2023-08-28 22:00:02 bioat.lib.libpandas.set_option: set pandas: display.width=120
INFO  @ 2023-08-28 22:00:02 bioat.lib.libpandas.set_option: set pandas: display.max_columns=None
INFO  @ 2023-08-28 22:00:02 bioat.lib.libpandas.set_option: set pandas: display.max_rows=50


# classification for Unassorted off-target

## get df_arts_unassorted
## if you are from step 12. start from here!!

In [2]:
# df_arts_unassorted = pd.read_csv('../art_seqinfo/df_arts_unassorted.csv')
df_arts_unassorted = pd.read_csv('../art_seqinfo/df_arts_unassorted_add_seq_v2.csv')
df_arts_unassorted.drop(columns='sequence', inplace=True)
df_arts_unassorted

chrom      start        end               region_index align_chr_name  align_chr_start  align_chr_end  \
0      chr7  123412912  123413069   chr7_123412962_123413019           chr7        123412967      123412978   
1     chr19   45742345   45742506    chr19_45742395_45742456          chr19         45742447       45742458   
2     chr11  113102877  113103036  chr11_113102927_113102986          chr11        113102970      113102981   
3      chr4  127523422  127523578   chr4_127523472_127523528           chr4        127523489      127523500   
4     chr22   19547402   19547545    chr22_19547452_19547495          chr22         19547450       19547461   
...     ...        ...        ...                        ...            ...              ...            ...   
8520  chr10  128384449  128384564  chr10_128384499_128384514          chr10        128384451      128384467   
8521  chr12   15347569   15347765    chr12_15347619_15347715          chr12         15347722       15347738   
8522   chr8   10421053   10421176     chr8_10421103_10421126           chr8         10421063       10421079   
8523   chr1  165880712  165880857   chr1_165880762_165880807           chr1        165880789      165880805   
8524   chr5  122273643  122273767   chr5_122273693_122273717           chr5        122273684      122273700   

     align_strand  align_dist_to_signal align_N0_base  align_total_match  align_total_mismatch  \
0               -                     5             A                  7                     5   
1               +                    -2             A                 10                     2   
2               +                     5             A                  9                     3   
3               +                   -28             T                  7                     5   
4               -                    -2             T                  7                     5   
...           ...                   ...           ...                ...                   ...   
8520            -                   -63             C                 10                     7   
8521            +                   -23             T                  8                     9   
8522            +                    24             C                  9                     8   
8523            +                   -43             C                  9                     8   
8524            -                   -33             C                  8                     9   

      align_degen_total_match  align_degen_total_mismatch  align_degen_num  align_total_gap  align_score  \
0                          12                           0                5                0         50.0   
1                          11                           1                1                0         49.0   
2                          11                           1                2                0         47.0   
3                          10                           2                3                0         46.0   
4                          10                           2                3                0         46.0   
...                       ...                         ...              ...              ...          ...   
8520                       10                           7                0                0         22.0   
8521                        9                           8                1                0         21.0   
8522                       10                           7                1                0         20.0   
8523                       10                           7                1                0         20.0   
8524                       10                           7                2                0         18.0   

       align_target_seq   align_info_state    align_query_seq                                file_info  \
0          ATAAACTTTAAT       ||...||||..|       ATGGGCTTTGGT  ../art/DetectSeq_ATP8-DddA11_R

In [3]:
index_unassorted = df_arts_unassorted['region_index'].unique().tolist()
index_unassorted[:10]

['chr7_123412962_123413019',
 'chr19_45742395_45742456',
 'chr11_113102927_113102986',
 'chr4_127523472_127523528',
 'chr22_19547452_19547495',
 'chr20_46523898_46523967',
 'chr21_43041285_43041353',
 'chr1_241226032_241226091',
 'chr2_59759874_59759954',
 'chr18_44726561_44726607']

## check if ok than all ctcf binding sites have been removed

In [4]:
df_ctcf = pd.read_csv(f'{HOME}/1.database/public_data/CTCF_ATAC_data/ENCFF285QVL_CTCF_binding_sites_fix_range.bed',
                      sep='\t')


coor_ctcf_bed = BedTool.from_dataframe(df_ctcf.iloc[:, :3])
coor_ctcf_bed.to_dataframe()

coor_unassorted = pd.Series(index_unassorted).str.split('_', expand=True)
coor_unassorted.columns = ['chrom', 'start', 'end']
coor_unassorted[['start', 'end']] = coor_unassorted[['start', 'end']].astype(int)
coor_unassorted_bed = BedTool.from_dataframe(coor_unassorted)
df_coor_intersect = coor_unassorted_bed.intersect(coor_ctcf_bed, loj=True).to_dataframe()
df_coor_intersect.columns = ['chrom1', 'start1', 'end1', 'chrom2', 'start2', 'end2']
df_coor_tas_indep = df_coor_intersect.query('start2 != -1').drop_duplicates(subset=['chrom1', 'start1', 'end1'])
df_coor_unassorted = df_coor_intersect.query('start2 == -1').drop_duplicates(subset=['chrom1', 'start1', 'end1'])
print(df_coor_tas_indep.empty)
df_coor_unassorted

True


chrom1     start1       end1 chrom2  start2  end2
0      chr7  123412962  123413019      .      -1    -1
1     chr19   45742395   45742456      .      -1    -1
2     chr11  113102927  113102986      .      -1    -1
3      chr4  127523472  127523528      .      -1    -1
4     chr22   19547452   19547495      .      -1    -1
...     ...        ...        ...    ...     ...   ...
8136   chr9    6664852    6664873      .      -1    -1
8137  chr18   69079311   69079386      .      -1    -1
8138   chr4   77820269   77820283      .      -1    -1
8139  chr17   76040058   76040077      .      -1    -1
8140  chr10  128384499  128384514      .      -1    -1

[8141 rows x 6 columns]

## from poission_res get highest site index

In [5]:
df_detect_seq_signal = pd.read_csv('../poisson_res/poisson_res_all.tsv.gz', sep='\t')

In [6]:
df_detect_seq_signal.columns

Index(['<sample>', 'chr_name', 'region_start', 'region_end', 'mpmat_index', 'region_site_num', 'region_block_site_num',
       'region_mut_site_num', 'region_site_index', 'region_block_state', 'region_highest_site_index',
       'region_highest_site_mut_num', 'region_highest_site_cover_num', 'region_highest_site_mut_ratio', 'ctrl_count',
       'treat_count', 'ctrl_mut_count', 'treat_mut_count', 'ctrl_count.norm', 'treat_count.norm',
       'ctrl_mut_count.norm', 'treat_mut_count.norm', 'count_info', 'log2_FC', 'log2_FC_mut', 'test_state', 'p_value',
       'FDR'],
      dtype='object')

In [7]:
df_detect_seq_signal_unassorted = df_detect_seq_signal[lambda df: df["mpmat_index"].map(lambda x: x in index_unassorted)][["mpmat_index", "treat_mut_count.norm", "<sample>", "region_highest_site_index"]].copy()

df_detect_seq_signal_unassorted

mpmat_index  treat_mut_count.norm                      <sample> region_highest_site_index
3             chr1_268547_268579              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA
18            chr1_816630_816666              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA
21            chr1_968672_968697              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA
43          chr1_1942305_1942315              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA
44          chr1_1942317_1942350              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT
...                          ...                   ...                           ...                       ...
705621  chrX_154924102_154924198              0.041798  DetectSeq_SIRT6-DddA11_REP-2         chrX_154924110_CT
705636  chrX_155216206_155216224              0.106817  DetectSeq_SIRT6-DddA11_REP-2         chrX_155216224_GA
705639  chrX_155439898_155439954              0.027865  DetectSeq_SIRT6-DddA11_REP-2         chrX_155439946_GA
705641  chrX_155570076_155570156              0.000000  DetectSeq_SIRT6-DddA11_REP-2         chrX_155570155_GA
705643  chrX_155612557_155612564              0.051086  DetectSeq_SIRT6-DddA11_REP-2         chrX_155612557_CT

[56987 rows x 4 columns]

In [8]:
df_detect_seq_signal_unassorted

mpmat_index  treat_mut_count.norm                      <sample> region_highest_site_index
3             chr1_268547_268579              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA
18            chr1_816630_816666              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA
21            chr1_968672_968697              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA
43          chr1_1942305_1942315              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA
44          chr1_1942317_1942350              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT
...                          ...                   ...                           ...                       ...
705621  chrX_154924102_154924198              0.041798  DetectSeq_SIRT6-DddA11_REP-2         chrX_154924110_CT
705636  chrX_155216206_155216224              0.106817  DetectSeq_SIRT6-DddA11_REP-2         chrX_155216224_GA
705639  chrX_155439898_155439954              0.027865  DetectSeq_SIRT6-DddA11_REP-2         chrX_155439946_GA
705641  chrX_155570076_155570156              0.000000  DetectSeq_SIRT6-DddA11_REP-2         chrX_155570155_GA
705643  chrX_155612557_155612564              0.051086  DetectSeq_SIRT6-DddA11_REP-2         chrX_155612557_CT

[56987 rows x 4 columns]

In [9]:
df_for_motif = df_detect_seq_signal_unassorted.drop_duplicates(subset=['mpmat_index', '<sample>'])
df_for_motif
# 没有组间duplicate

mpmat_index  treat_mut_count.norm                      <sample> region_highest_site_index
3             chr1_268547_268579              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA
18            chr1_816630_816666              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA
21            chr1_968672_968697              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA
43          chr1_1942305_1942315              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA
44          chr1_1942317_1942350              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT
...                          ...                   ...                           ...                       ...
705621  chrX_154924102_154924198              0.041798  DetectSeq_SIRT6-DddA11_REP-2         chrX_154924110_CT
705636  chrX_155216206_155216224              0.106817  DetectSeq_SIRT6-DddA11_REP-2         chrX_155216224_GA
705639  chrX_155439898_155439954              0.027865  DetectSeq_SIRT6-DddA11_REP-2         chrX_155439946_GA
705641  chrX_155570076_155570156              0.000000  DetectSeq_SIRT6-DddA11_REP-2         chrX_155570155_GA
705643  chrX_155612557_155612564              0.051086  DetectSeq_SIRT6-DddA11_REP-2         chrX_155612557_CT

[56987 rows x 4 columns]

In [10]:
df_for_motif = df_for_motif.drop_duplicates(subset=['mpmat_index'])
df_for_motif

mpmat_index  treat_mut_count.norm                     <sample> region_highest_site_index
3             chr1_268547_268579              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA
18            chr1_816630_816666              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA
21            chr1_968672_968697              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA
43          chr1_1942305_1942315              0.000000  DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA
44          chr1_1942317_1942350              0.000000  DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT
...                          ...                   ...                          ...                       ...
100779  chrX_154924102_154924198              0.029804  DetectSeq_ATP8-DddA11_REP-1         chrX_154924125_CT
100794  chrX_155216206_155216224              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155216206_GA
100797  chrX_155439898_155439954              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155439915_GA
100799  chrX_155570076_155570156              0.059608  DetectSeq_ATP8-DddA11_REP-1         chrX_155570130_GA
100801  chrX_155612557_155612564              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155612557_CT

[8141 rows x 4 columns]

In [11]:
df_for_motif = df_for_motif.drop_duplicates(subset=['region_highest_site_index'])
df_for_motif

mpmat_index  treat_mut_count.norm                     <sample> region_highest_site_index
3             chr1_268547_268579              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA
18            chr1_816630_816666              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA
21            chr1_968672_968697              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA
43          chr1_1942305_1942315              0.000000  DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA
44          chr1_1942317_1942350              0.000000  DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT
...                          ...                   ...                          ...                       ...
100779  chrX_154924102_154924198              0.029804  DetectSeq_ATP8-DddA11_REP-1         chrX_154924125_CT
100794  chrX_155216206_155216224              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155216206_GA
100797  chrX_155439898_155439954              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155439915_GA
100799  chrX_155570076_155570156              0.059608  DetectSeq_ATP8-DddA11_REP-1         chrX_155570130_GA
100801  chrX_155612557_155612564              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155612557_CT

[8141 rows x 4 columns]

In [12]:
df_for_motif['chrom'] = df_for_motif['region_highest_site_index'].str.split('_').str[0]
df_for_motif['chromStart'] = df_for_motif['region_highest_site_index'].str.split('_').str[1].astype(int) - 200
df_for_motif['chromEnd'] = df_for_motif['region_highest_site_index'].str.split('_').str[1].astype(int) + 200

In [13]:
df_for_motif

mpmat_index  treat_mut_count.norm                     <sample> region_highest_site_index chrom  \
3             chr1_268547_268579              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA  chr1   
18            chr1_816630_816666              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA  chr1   
21            chr1_968672_968697              0.000000  DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA  chr1   
43          chr1_1942305_1942315              0.000000  DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA  chr1   
44          chr1_1942317_1942350              0.000000  DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT  chr1   
...                          ...                   ...                          ...                       ...   ...   
100779  chrX_154924102_154924198              0.029804  DetectSeq_ATP8-DddA11_REP-1         chrX_154924125_CT  chrX   
100794  chrX_155216206_155216224              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155216206_GA  chrX   
100797  chrX_155439898_155439954              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155439915_GA  chrX   
100799  chrX_155570076_155570156              0.059608  DetectSeq_ATP8-DddA11_REP-1         chrX_155570130_GA  chrX   
100801  chrX_155612557_155612564              0.000000  DetectSeq_ATP8-DddA11_REP-1         chrX_155612557_CT  chrX   

        chromStart   chromEnd  
3           268347     268747  
18          816430     816830  
21          968472     968872  
43         1942105    1942505  
44         1942117    1942517  
...            ...        ...  
100779   154923925  154924325  
100794   155216006  155216406  
100797   155439715  155440115  
100799   155569930  155570330  
100801   155612357  155612757  

[8141 rows x 7 columns]

In [14]:
df_coor_unassorted = df_for_motif[['chrom', "chromStart", "chromEnd"]].copy()
df_coor_unassorted

chrom  chromStart   chromEnd
3       chr1      268347     268747
18      chr1      816430     816830
21      chr1      968472     968872
43      chr1     1942105    1942505
44      chr1     1942117    1942517
...      ...         ...        ...
100779  chrX   154923925  154924325
100794  chrX   155216006  155216406
100797  chrX   155439715  155440115
100799  chrX   155569930  155570330
100801  chrX   155612357  155612757

[8141 rows x 3 columns]

In [15]:
bed_unassorted = BedTool().from_dataframe(df_coor_unassorted)
print(bed_unassorted.head())
len(bed_unassorted)

chr1	268347	268747
 chr1	816430	816830
 chr1	968472	968872
 chr1	1942105	1942505
 chr1	1942117	1942517
 chr1	2363166	2363566
 chr1	2549366	2549766
 chr1	3392811	3393211
 chr1	3614742	3615142
 chr1	3656826	3657226
 None


8141

In [16]:
# 指定参考基因组序列信息，并从中获取每个序列信息
fasta_unassorted = bed_unassorted.getfasta(
    fi=f'{HOME}/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa')
fasta_unassorted

<BedTool(/var/folders/79/yd907p9x4537vqhgfrm0md680000gn/T/pybedtools.bmdihacu.tmp)>

In [17]:
# 将region序列信息转为dataframe
df_seqinfo_unassorted = pd.DataFrame(fasta_unassorted.print_sequence().split('>')[1:], columns=['info'])
df_seqinfo_unassorted[['coordinate', 'sequence']] = df_seqinfo_unassorted['info'].str.strip().str.split('\n', expand=True)
df_seqinfo_unassorted

info                coordinate                                 sequence
0     chr1:268347-268747\nggttcctaaataagat...        chr1:268347-268747  ggttcctaaataagatagctattacacaaagataaa...
1     chr1:816430-816830\ngggtgtggttacaagg...        chr1:816430-816830  gggtgtggttacaaggatgtttcctttataaAAATG...
2     chr1:968472-968872\nGGAGTCCCCTCCCCTC...        chr1:968472-968872  GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3     chr1:1942105-1942505\nGCCCTCAGCTCCTT...      chr1:1942105-1942505  GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...
4     chr1:1942117-1942517\nTTGCACACCTCGAA...      chr1:1942117-1942517  TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...
...                                       ...                       ...                                      ...
8136  chrX:154923925-154924325\ncacgccactg...  chrX:154923925-154924325  cacgccactgactccagcctgcgtgacagagcaaga...
8137  chrX:155216006-155216406\nGAAGCTCCCT...  chrX:155216006-155216406  GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...
8138  chrX:155439715-155440115\ngtgaaatgac...  chrX:155439715-155440115  gtgaaatgacaggaaacaaagcaaggaaagggaggg...
8139  chrX:155569930-155570330\ncgagcaaaat...  chrX:155569930-155570330  cgagcaaaataaccagctaacatcataatgacagga...
8140  chrX:155612357-155612757\nccactctact...  chrX:155612357-155612757  ccactctactgccgaacaaagtgaggcaggaagagg...

[8141 rows x 3 columns]

In [18]:
# 去掉序列中含有N的 region（可能坐落于端粒或中性粒，影响分析）
df_seqinfo_unassorted = df_seqinfo_unassorted.query('~sequence.str.upper().str.contains("N")').copy()
df_seqinfo_unassorted

info                coordinate                                 sequence
0     chr1:268347-268747\nggttcctaaataagat...        chr1:268347-268747  ggttcctaaataagatagctattacacaaagataaa...
1     chr1:816430-816830\ngggtgtggttacaagg...        chr1:816430-816830  gggtgtggttacaaggatgtttcctttataaAAATG...
2     chr1:968472-968872\nGGAGTCCCCTCCCCTC...        chr1:968472-968872  GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3     chr1:1942105-1942505\nGCCCTCAGCTCCTT...      chr1:1942105-1942505  GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...
4     chr1:1942117-1942517\nTTGCACACCTCGAA...      chr1:1942117-1942517  TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...
...                                       ...                       ...                                      ...
8136  chrX:154923925-154924325\ncacgccactg...  chrX:154923925-154924325  cacgccactgactccagcctgcgtgacagagcaaga...
8137  chrX:155216006-155216406\nGAAGCTCCCT...  chrX:155216006-155216406  GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...
8138  chrX:155439715-155440115\ngtgaaatgac...  chrX:155439715-155440115  gtgaaatgacaggaaacaaagcaaggaaagggaggg...
8139  chrX:155569930-155570330\ncgagcaaaat...  chrX:155569930-155570330  cgagcaaaataaccagctaacatcataatgacagga...
8140  chrX:155612357-155612757\nccactctact...  chrX:155612357-155612757  ccactctactgccgaacaaagtgaggcaggaagagg...

[8141 rows x 3 columns]

In [19]:
# 整理dataframe
df_seqinfo_unassorted[['chrom', 'chromStart', 'chromEnd']] = (
    df_seqinfo_unassorted['coordinate']
    .str.replace(':', '\t')
    .str.replace('-', '\t')
    .str.split('\t', expand=True)
)
df_seqinfo_unassorted

info                coordinate                                 sequence  \
0     chr1:268347-268747\nggttcctaaataagat...        chr1:268347-268747  ggttcctaaataagatagctattacacaaagataaa...   
1     chr1:816430-816830\ngggtgtggttacaagg...        chr1:816430-816830  gggtgtggttacaaggatgtttcctttataaAAATG...   
2     chr1:968472-968872\nGGAGTCCCCTCCCCTC...        chr1:968472-968872  GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...   
3     chr1:1942105-1942505\nGCCCTCAGCTCCTT...      chr1:1942105-1942505  GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...   
4     chr1:1942117-1942517\nTTGCACACCTCGAA...      chr1:1942117-1942517  TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...   
...                                       ...                       ...                                      ...   
8136  chrX:154923925-154924325\ncacgccactg...  chrX:154923925-154924325  cacgccactgactccagcctgcgtgacagagcaaga...   
8137  chrX:155216006-155216406\nGAAGCTCCCT...  chrX:155216006-155216406  GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...   
8138  chrX:155439715-155440115\ngtgaaatgac...  chrX:155439715-155440115  gtgaaatgacaggaaacaaagcaaggaaagggaggg...   
8139  chrX:155569930-155570330\ncgagcaaaat...  chrX:155569930-155570330  cgagcaaaataaccagctaacatcataatgacagga...   
8140  chrX:155612357-155612757\nccactctact...  chrX:155612357-155612757  ccactctactgccgaacaaagtgaggcaggaagagg...   

     chrom chromStart   chromEnd  
0     chr1     268347     268747  
1     chr1     816430     816830  
2     chr1     968472     968872  
3     chr1    1942105    1942505  
4     chr1    1942117    1942517  
...    ...        ...        ...  
8136  chrX  154923925  154924325  
8137  chrX  155216006  155216406  
8138  chrX  155439715  155440115  
8139  chrX  155569930  155570330  
8140  chrX  155612357  155612757  

[8141 rows x 6 columns]

In [20]:
df_seqinfo_unassorted[['chromStart', 'chromEnd']] = df_seqinfo_unassorted[['chromStart', 'chromEnd']].astype(int)
df_seqinfo_unassorted = df_seqinfo_unassorted[['chrom', 'chromStart', 'chromEnd', 'sequence']].copy()
df_seqinfo_unassorted['sequence'] = df_seqinfo_unassorted['sequence'].str.upper()
df_seqinfo_unassorted

chrom  chromStart   chromEnd                                 sequence
0     chr1      268347     268747  GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...
1     chr1      816430     816830  GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...
2     chr1      968472     968872  GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...
3     chr1     1942105    1942505  GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...
4     chr1     1942117    1942517  TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...
...    ...         ...        ...                                      ...
8136  chrX   154923925  154924325  CACGCCACTGACTCCAGCCTGCGTGACAGAGCAAGA...
8137  chrX   155216006  155216406  GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...
8138  chrX   155439715  155440115  GTGAAATGACAGGAAACAAAGCAAGGAAAGGGAGGG...
8139  chrX   155569930  155570330  CGAGCAAAATAACCAGCTAACATCATAATGACAGGA...
8140  chrX   155612357  155612757  CCACTCTACTGCCGAACAAAGTGAGGCAGGAAGAGG...

[8141 rows x 4 columns]

In [21]:
df_seqinfo_unassorted.isna().sum()

chrom         0
chromStart    0
chromEnd      0
sequence      0
dtype: int64

In [22]:
df_seqinfo_unassorted['idx'] = df_seqinfo_unassorted['chrom'] + '_' + (df_seqinfo_unassorted['chromStart'] + 200).astype(str)
df_seqinfo_unassorted

chrom  chromStart   chromEnd                                 sequence             idx
0     chr1      268347     268747  GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...     chr1_268547
1     chr1      816430     816830  GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...     chr1_816630
2     chr1      968472     968872  GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...     chr1_968672
3     chr1     1942105    1942505  GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...    chr1_1942305
4     chr1     1942117    1942517  TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...    chr1_1942317
...    ...         ...        ...                                      ...             ...
8136  chrX   154923925  154924325  CACGCCACTGACTCCAGCCTGCGTGACAGAGCAAGA...  chrX_154924125
8137  chrX   155216006  155216406  GAAGCTCCCTGGGATGGTGAGACCAGAAAGGCACAG...  chrX_155216206
8138  chrX   155439715  155440115  GTGAAATGACAGGAAACAAAGCAAGGAAAGGGAGGG...  chrX_155439915
8139  chrX   155569930  155570330  CGAGCAAAATAACCAGCTAACATCATAATGACAGGA...  chrX_155570130
8140  chrX   155612357  155612757  CCACTCTACTGCCGAACAAAGTGAGGCAGGAAGAGG...  chrX_155612557

[8141 rows x 5 columns]

In [23]:
df_detect_seq_signal_unassorted['idx'] = df_detect_seq_signal_unassorted['region_highest_site_index'].str[:-3]
df_detect_seq_signal_unassorted

mpmat_index  treat_mut_count.norm                      <sample> region_highest_site_index  \
3             chr1_268547_268579              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA   
18            chr1_816630_816666              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA   
21            chr1_968672_968697              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA   
43          chr1_1942305_1942315              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA   
44          chr1_1942317_1942350              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT   
...                          ...                   ...                           ...                       ...   
705621  chrX_154924102_154924198              0.041798  DetectSeq_SIRT6-DddA11_REP-2         chrX_154924110_CT   
705636  chrX_155216206_155216224              0.106817  DetectSeq_SIRT6-DddA11_REP-2         chrX_155216224_GA   
705639  chrX_155439898_155439954              0.027865  DetectSeq_SIRT6-DddA11_REP-2         chrX_155439946_GA   
705641  chrX_155570076_155570156              0.000000  DetectSeq_SIRT6-DddA11_REP-2         chrX_155570155_GA   
705643  chrX_155612557_155612564              0.051086  DetectSeq_SIRT6-DddA11_REP-2         chrX_155612557_CT   

                   idx  
3          chr1_268547  
18         chr1_816630  
21         chr1_968672  
43        chr1_1942305  
44        chr1_1942317  
...                ...  
705621  chrX_154924110  
705636  chrX_155216224  
705639  chrX_155439946  
705641  chrX_155570155  
705643  chrX_155612557  

[56987 rows x 5 columns]

In [24]:
df_for_motif_unassorted_add_seq = pd.merge(df_detect_seq_signal_unassorted, df_seqinfo_unassorted, on='idx', how='left')
df_for_motif_unassorted_add_seq

mpmat_index  treat_mut_count.norm                      <sample> region_highest_site_index  \
0            chr1_268547_268579              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_268547_GA   
1            chr1_816630_816666              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_816630_GA   
2            chr1_968672_968697              0.000000   DetectSeq_ATP8-DddA11_REP-1            chr1_968672_GA   
3          chr1_1942305_1942315              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942305_GA   
4          chr1_1942317_1942350              0.000000   DetectSeq_ATP8-DddA11_REP-1           chr1_1942317_CT   
...                         ...                   ...                           ...                       ...   
56982  chrX_154924102_154924198              0.041798  DetectSeq_SIRT6-DddA11_REP-2         chrX_154924110_CT   
56983  chrX_155216206_155216224              0.106817  DetectSeq_SIRT6-DddA11_REP-2         chrX_155216224_GA   
56984  chrX_155439898_155439954              0.027865  DetectSeq_SIRT6-DddA11_REP-2         chrX_155439946_GA   
56985  chrX_155570076_155570156              0.000000  DetectSeq_SIRT6-DddA11_REP-2         chrX_155570155_GA   
56986  chrX_155612557_155612564              0.051086  DetectSeq_SIRT6-DddA11_REP-2         chrX_155612557_CT   

                  idx chrom   chromStart     chromEnd                                 sequence  
0         chr1_268547  chr1     268347.0     268747.0  GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...  
1         chr1_816630  chr1     816430.0     816830.0  GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...  
2         chr1_968672  chr1     968472.0     968872.0  GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...  
3        chr1_1942305  chr1    1942105.0    1942505.0  GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...  
4        chr1_1942317  chr1    1942117.0    1942517.0  TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...  
...               ...   ...          ...          ...                                      ...  
56982  chrX_154924110   NaN          NaN          NaN                                      NaN  
56983  chrX_155216224   NaN          NaN          NaN                                      NaN  
56984  chrX_155439946   NaN          NaN          NaN                                      NaN  
56985  chrX_155570155   NaN          NaN          NaN                                      NaN  
56986  chrX_155612557  chrX  155612357.0  155612757.0  CCACTCTACTGCCGAACAAAGTGAGGCAGGAAGAGG...  

[56987 rows x 9 columns]

In [25]:
df_for_motif_unassorted_add_seq.isna().sum()

mpmat_index                      0
treat_mut_count.norm             0
<sample>                         0
region_highest_site_index        0
idx                              0
chrom                        33256
chromStart                   33256
chromEnd                     33256
sequence                     33256
dtype: int64

In [26]:
df_for_motif_unassorted_add_seq = df_for_motif_unassorted_add_seq[['mpmat_index', 'treat_mut_count.norm', '<sample>', 'sequence']].copy()
df_for_motif_unassorted_add_seq

mpmat_index  treat_mut_count.norm                      <sample>  \
0            chr1_268547_268579              0.000000   DetectSeq_ATP8-DddA11_REP-1   
1            chr1_816630_816666              0.000000   DetectSeq_ATP8-DddA11_REP-1   
2            chr1_968672_968697              0.000000   DetectSeq_ATP8-DddA11_REP-1   
3          chr1_1942305_1942315              0.000000   DetectSeq_ATP8-DddA11_REP-1   
4          chr1_1942317_1942350              0.000000   DetectSeq_ATP8-DddA11_REP-1   
...                         ...                   ...                           ...   
56982  chrX_154924102_154924198              0.041798  DetectSeq_SIRT6-DddA11_REP-2   
56983  chrX_155216206_155216224              0.106817  DetectSeq_SIRT6-DddA11_REP-2   
56984  chrX_155439898_155439954              0.027865  DetectSeq_SIRT6-DddA11_REP-2   
56985  chrX_155570076_155570156              0.000000  DetectSeq_SIRT6-DddA11_REP-2   
56986  chrX_155612557_155612564              0.051086  DetectSeq_SIRT6-DddA11_REP-2   

                                      sequence  
0      GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...  
1      GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...  
2      GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...  
3      GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...  
4      TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...  
...                                        ...  
56982                                      NaN  
56983                                      NaN  
56984                                      NaN  
56985                                      NaN  
56986  CCACTCTACTGCCGAACAAAGTGAGGCAGGAAGAGG...  

[56987 rows x 4 columns]

In [27]:
df_for_motif_unassorted_add_seq = df_for_motif_unassorted_add_seq.query('sequence.notna()')
df_for_motif_unassorted_add_seq

mpmat_index  treat_mut_count.norm                      <sample>  \
0            chr1_268547_268579              0.000000   DetectSeq_ATP8-DddA11_REP-1   
1            chr1_816630_816666              0.000000   DetectSeq_ATP8-DddA11_REP-1   
2            chr1_968672_968697              0.000000   DetectSeq_ATP8-DddA11_REP-1   
3          chr1_1942305_1942315              0.000000   DetectSeq_ATP8-DddA11_REP-1   
4          chr1_1942317_1942350              0.000000   DetectSeq_ATP8-DddA11_REP-1   
...                         ...                   ...                           ...   
56964  chrX_139884820_139884879              0.000000  DetectSeq_SIRT6-DddA11_REP-2   
56968  chrX_144511720_144511761              0.078952  DetectSeq_SIRT6-DddA11_REP-2   
56969  chrX_149297009_149297023              0.060375  DetectSeq_SIRT6-DddA11_REP-2   
56973  chrX_150967655_150967689              0.069663  DetectSeq_SIRT6-DddA11_REP-2   
56986  chrX_155612557_155612564              0.051086  DetectSeq_SIRT6-DddA11_REP-2   

                                      sequence  
0      GGTTCCTAAATAAGATAGCTATTACACAAAGATAAA...  
1      GGGTGTGGTTACAAGGATGTTTCCTTTATAAAAATG...  
2      GGAGTCCCCTCCCCTCTCAGCACATGCAGTGATACC...  
3      GCCCTCAGCTCCTTGCACACCTCGAACTTCAGGCGC...  
4      TTGCACACCTCGAACTTCAGGCGCAGGGCAGCTTTC...  
...                                        ...  
56964  CTCAGCAGTACTTTTTAAAATTTTCCAAATTTGCAA...  
56968  AATACACTAGAAGCTCAATCCCCACCAGTATGTCAT...  
56969  TTCATTGAGATGGGAACTGTGCAATGCTGCACTGAA...  
56973  CACTACAAGACCTTAGAATCGCAGCGTCTTCATCCA...  
56986  CCACTCTACTGCCGAACAAAGTGAGGCAGGAAGAGG...  

[23731 rows x 4 columns]

In [28]:
df_for_motif_unassorted_add_seq = df_for_motif_unassorted_add_seq.sort_values('mpmat_index').drop_duplicates(subset='mpmat_index')[['mpmat_index', 'sequence']]
df_for_motif_unassorted_add_seq

mpmat_index                                 sequence
21398  chr10_100175182_100175206  ACCAAAGAGGACTCAATTGGTAGATTTTAAATTGAT...
5117   chr10_100229657_100229680  GCCGCCGGTGCCCAGCCGCTCCCGCATCTCCCAGGG...
37682  chr10_100373544_100373555  GAATGGAGAGGAAGCGCCTCCCTCTCCACCCACTCT...
37683  chr10_100793730_100793754  ACAGGTAACCTGGGGGCTCTTTTTCGGGGGAGCTAA...
21402  chr10_101189822_101189858  TCTGGTCCAGCTACCTCCCACCTCGCCTTCCTTGAC...
...                          ...                                      ...
7998      chrX_95825682_95825736  GGCCAAAATCCAGAACACTGACATGGTGGCAAAGAT...
16140     chrX_96177233_96177267  TCTTGGAAACAAGCTAAATCAATATGTCATGCAAAA...
8000      chrX_96252440_96252507  ATTTATCTTTGTAGAGACAGGGTCTCACTATGTTGG...
24283     chrX_97027605_97027643  CAACAGCCAAAAGATTGTTTGCAATAAAGACCTTAT...
48707     chrX_98001743_98001757  CACTGTAATTAAGTGGGGGAAAGAATAATTTTATTA...

[8141 rows x 2 columns]

In [29]:
df_arts_unassorted_add_seq = pd.merge(df_arts_unassorted, df_for_motif_unassorted_add_seq, left_on='region_index', right_on='mpmat_index', how='left')
df_arts_unassorted_add_seq

chrom      start        end               region_index align_chr_name  align_chr_start  align_chr_end  \
0      chr7  123412912  123413069   chr7_123412962_123413019           chr7        123412967      123412978   
1     chr19   45742345   45742506    chr19_45742395_45742456          chr19         45742447       45742458   
2     chr11  113102877  113103036  chr11_113102927_113102986          chr11        113102970      113102981   
3      chr4  127523422  127523578   chr4_127523472_127523528           chr4        127523489      127523500   
4     chr22   19547402   19547545    chr22_19547452_19547495          chr22         19547450       19547461   
...     ...        ...        ...                        ...            ...              ...            ...   
8520  chr10  128384449  128384564  chr10_128384499_128384514          chr10        128384451      128384467   
8521  chr12   15347569   15347765    chr12_15347619_15347715          chr12         15347722       15347738   
8522   chr8   10421053   10421176     chr8_10421103_10421126           chr8         10421063       10421079   
8523   chr1  165880712  165880857   chr1_165880762_165880807           chr1        165880789      165880805   
8524   chr5  122273643  122273767   chr5_122273693_122273717           chr5        122273684      122273700   

     align_strand  align_dist_to_signal align_N0_base  align_total_match  align_total_mismatch  \
0               -                     5             A                  7                     5   
1               +                    -2             A                 10                     2   
2               +                     5             A                  9                     3   
3               +                   -28             T                  7                     5   
4               -                    -2             T                  7                     5   
...           ...                   ...           ...                ...                   ...   
8520            -                   -63             C                 10                     7   
8521            +                   -23             T                  8                     9   
8522            +                    24             C                  9                     8   
8523            +                   -43             C                  9                     8   
8524            -                   -33             C                  8                     9   

      align_degen_total_match  align_degen_total_mismatch  align_degen_num  align_total_gap  align_score  \
0                          12                           0                5                0         50.0   
1                          11                           1                1                0         49.0   
2                          11                           1                2                0         47.0   
3                          10                           2                3                0         46.0   
4                          10                           2                3                0         46.0   
...                       ...                         ...              ...              ...          ...   
8520                       10                           7                0                0         22.0   
8521                        9                           8                1                0         21.0   
8522                       10                           7                1                0         20.0   
8523                       10                           7                1                0         20.0   
8524                       10                           7                2                0         18.0   

       align_target_seq   align_info_state    align_query_seq                                file_info  \
0          ATAAACTTTAAT       ||...||||..|       ATGGGCTTTGGT  ../art/DetectSeq_ATP8-DddA11_R

In [30]:
df_arts_unassorted_add_seq.drop(columns='mpmat_index', inplace=True)
df_arts_unassorted_add_seq

chrom      start        end               region_index align_chr_name  align_chr_start  align_chr_end  \
0      chr7  123412912  123413069   chr7_123412962_123413019           chr7        123412967      123412978   
1     chr19   45742345   45742506    chr19_45742395_45742456          chr19         45742447       45742458   
2     chr11  113102877  113103036  chr11_113102927_113102986          chr11        113102970      113102981   
3      chr4  127523422  127523578   chr4_127523472_127523528           chr4        127523489      127523500   
4     chr22   19547402   19547545    chr22_19547452_19547495          chr22         19547450       19547461   
...     ...        ...        ...                        ...            ...              ...            ...   
8520  chr10  128384449  128384564  chr10_128384499_128384514          chr10        128384451      128384467   
8521  chr12   15347569   15347765    chr12_15347619_15347715          chr12         15347722       15347738   
8522   chr8   10421053   10421176     chr8_10421103_10421126           chr8         10421063       10421079   
8523   chr1  165880712  165880857   chr1_165880762_165880807           chr1        165880789      165880805   
8524   chr5  122273643  122273767   chr5_122273693_122273717           chr5        122273684      122273700   

     align_strand  align_dist_to_signal align_N0_base  align_total_match  align_total_mismatch  \
0               -                     5             A                  7                     5   
1               +                    -2             A                 10                     2   
2               +                     5             A                  9                     3   
3               +                   -28             T                  7                     5   
4               -                    -2             T                  7                     5   
...           ...                   ...           ...                ...                   ...   
8520            -                   -63             C                 10                     7   
8521            +                   -23             T                  8                     9   
8522            +                    24             C                  9                     8   
8523            +                   -43             C                  9                     8   
8524            -                   -33             C                  8                     9   

      align_degen_total_match  align_degen_total_mismatch  align_degen_num  align_total_gap  align_score  \
0                          12                           0                5                0         50.0   
1                          11                           1                1                0         49.0   
2                          11                           1                2                0         47.0   
3                          10                           2                3                0         46.0   
4                          10                           2                3                0         46.0   
...                       ...                         ...              ...              ...          ...   
8520                       10                           7                0                0         22.0   
8521                        9                           8                1                0         21.0   
8522                       10                           7                1                0         20.0   
8523                       10                           7                1                0         20.0   
8524                       10                           7                2                0         18.0   

       align_target_seq   align_info_state    align_query_seq                                file_info  \
0          ATAAACTTTAAT       ||...||||..|       ATGGGCTTTGGT  ../art/DetectSeq_ATP8-DddA11_R

In [31]:
df_arts_unassorted_add_seq.isna().sum()

chrom                         0
start                         0
end                           0
region_index                  0
align_chr_name                0
align_chr_start               0
align_chr_end                 0
align_strand                  0
align_dist_to_signal          0
align_N0_base                 0
align_total_match             0
align_total_mismatch          0
align_degen_total_match       0
align_degen_total_mismatch    0
align_degen_num               0
align_total_gap               0
align_score                   0
align_target_seq              0
align_info_state              0
align_query_seq               0
file_info                     0
<sample>                      0
TAS-Dep_stat                  0
id                            0
sequence                      0
dtype: int64

In [32]:
df_arts_unassorted_add_seq.groupby(['<sample>', 'TAS-Dep_stat'])['region_index'].count()

<sample>                      TAS-Dep_stat    
DetectSeq_ATP8-DddA11_REP-1   Left                  83
                              Right                 15
DetectSeq_ATP8-DddA6_REP-1    Left                 477
                              Right                182
DetectSeq_ATP8-DddAwt_REP-1   Left                  10
                              Right                  3
DetectSeq_JAK2-DddA11_REP-1   Left                  44
                              Left_additional        4
                              Right                 37
                              Right_additional       1
DetectSeq_JAK2-DddA11_REP-2   Left                  85
                              Left_additional        7
                              Right                117
                              Right_additional      59
DetectSeq_SIRT6-DddA11_REP-1  Left                1149
                              Left_additional        6
                              Right                324
DetectSeq_SIRT6-Dd

In [33]:
os.makedirs('../art_seqinfo/unassorted', exist_ok=True)

In [34]:
df_arts_unassorted_add_seq = df_arts_unassorted_add_seq.query('~`TAS-Dep_stat`.str.contains("on-target")')
df_arts_unassorted_add_seq

chrom      start        end               region_index align_chr_name  align_chr_start  align_chr_end  \
0      chr7  123412912  123413069   chr7_123412962_123413019           chr7        123412967      123412978   
1     chr19   45742345   45742506    chr19_45742395_45742456          chr19         45742447       45742458   
2     chr11  113102877  113103036  chr11_113102927_113102986          chr11        113102970      113102981   
3      chr4  127523422  127523578   chr4_127523472_127523528           chr4        127523489      127523500   
4     chr22   19547402   19547545    chr22_19547452_19547495          chr22         19547450       19547461   
...     ...        ...        ...                        ...            ...              ...            ...   
8520  chr10  128384449  128384564  chr10_128384499_128384514          chr10        128384451      128384467   
8521  chr12   15347569   15347765    chr12_15347619_15347715          chr12         15347722       15347738   
8522   chr8   10421053   10421176     chr8_10421103_10421126           chr8         10421063       10421079   
8523   chr1  165880712  165880857   chr1_165880762_165880807           chr1        165880789      165880805   
8524   chr5  122273643  122273767   chr5_122273693_122273717           chr5        122273684      122273700   

     align_strand  align_dist_to_signal align_N0_base  align_total_match  align_total_mismatch  \
0               -                     5             A                  7                     5   
1               +                    -2             A                 10                     2   
2               +                     5             A                  9                     3   
3               +                   -28             T                  7                     5   
4               -                    -2             T                  7                     5   
...           ...                   ...           ...                ...                   ...   
8520            -                   -63             C                 10                     7   
8521            +                   -23             T                  8                     9   
8522            +                    24             C                  9                     8   
8523            +                   -43             C                  9                     8   
8524            -                   -33             C                  8                     9   

      align_degen_total_match  align_degen_total_mismatch  align_degen_num  align_total_gap  align_score  \
0                          12                           0                5                0         50.0   
1                          11                           1                1                0         49.0   
2                          11                           1                2                0         47.0   
3                          10                           2                3                0         46.0   
4                          10                           2                3                0         46.0   
...                       ...                         ...              ...              ...          ...   
8520                       10                           7                0                0         22.0   
8521                        9                           8                1                0         21.0   
8522                       10                           7                1                0         20.0   
8523                       10                           7                1                0         20.0   
8524                       10                           7                2                0         18.0   

       align_target_seq   align_info_state    align_query_seq                                file_info  \
0          ATAAACTTTAAT       ||...||||..|       ATGGGCTTTGGT  ../art/DetectSeq_ATP8-DddA11_R

In [35]:
ls = []
for sample, df in df_arts_unassorted_add_seq.groupby('<sample>'):
    # 上面的一个cell中的代码已经证明了duplication是TAS-Dep_stat alignment过程引入的，简单去重即可
    df = df.drop_duplicates(subset=['region_index'], keep='first')
    ls.append(df)
df_arts_unassorted_add_seq = pd.concat(ls)

# df_arts_unassorted_add_seq.to_csv('../art_seqinfo/df_arts_unassorted_add_seq.csv', index=False)
df_arts_unassorted_add_seq.to_csv('../art_seqinfo/df_arts_unassorted_add_seq_v3.csv', index=False)

In [36]:
for sample, df in df_arts_unassorted_add_seq.groupby('<sample>'):
    # 上面的duplication都是0则说明duplication是TAS-Dep_stat alignment过程引入的，简单去重即可
    print(f'df.shape[0] = {df.shape[0]}')
    print('查看是否有duplication', df.duplicated(subset='region_index').sum())

print('=' * 20)
for g, df in df_arts_unassorted_add_seq.groupby(['<sample>', 'TAS-Dep_stat']):
    sample, dep_stat = g
    print(f'df.shape[0] = {df.shape[0]}')
    print('查看是否有duplication', df.duplicated(subset='region_index').sum())
    # 上面的duplication都是0则说明duplication是TAS-Dep_stat alignment过程引入的，简单去重即可


df.shape[0] = 98
查看是否有duplication 0
df.shape[0] = 659
查看是否有duplication 0
df.shape[0] = 13
查看是否有duplication 0
df.shape[0] = 86
查看是否有duplication 0
df.shape[0] = 268
查看是否有duplication 0
df.shape[0] = 1479
查看是否有duplication 0
df.shape[0] = 5922
查看是否有duplication 0
df.shape[0] = 83
查看是否有duplication 0
df.shape[0] = 15
查看是否有duplication 0
df.shape[0] = 477
查看是否有duplication 0
df.shape[0] = 182
查看是否有duplication 0
df.shape[0] = 10
查看是否有duplication 0
df.shape[0] = 3
查看是否有duplication 0
df.shape[0] = 44
查看是否有duplication 0
df.shape[0] = 4
查看是否有duplication 0
df.shape[0] = 37
查看是否有duplication 0
df.shape[0] = 1
查看是否有duplication 0
df.shape[0] = 85
查看是否有duplication 0
df.shape[0] = 7
查看是否有duplication 0
df.shape[0] = 117
查看是否有duplication 0
df.shape[0] = 59
查看是否有duplication 0
df.shape[0] = 1149
查看是否有duplication 0
df.shape[0] = 6
查看是否有duplication 0
df.shape[0] = 324
查看是否有duplication 0
df.shape[0] = 4942
查看是否有duplication 0
df.shape[0] = 38
查看是否有duplication 0
df.shape[0] = 939
查看是否有duplication 0
df.shape[0] = 3
查看

In [37]:
df_arts_unassorted_add_seq

chrom      start        end               region_index align_chr_name  align_chr_start  align_chr_end  \
0      chr7  123412912  123413069   chr7_123412962_123413019           chr7        123412967      123412978   
1     chr19   45742345   45742506    chr19_45742395_45742456          chr19         45742447       45742458   
2     chr11  113102877  113103036  chr11_113102927_113102986          chr11        113102970      113102981   
3      chr4  127523422  127523578   chr4_127523472_127523528           chr4        127523489      127523500   
4     chr22   19547402   19547545    chr22_19547452_19547495          chr22         19547450       19547461   
...     ...        ...        ...                        ...            ...              ...            ...   
8520  chr10  128384449  128384564  chr10_128384499_128384514          chr10        128384451      128384467   
8521  chr12   15347569   15347765    chr12_15347619_15347715          chr12         15347722       15347738   
8522   chr8   10421053   10421176     chr8_10421103_10421126           chr8         10421063       10421079   
8523   chr1  165880712  165880857   chr1_165880762_165880807           chr1        165880789      165880805   
8524   chr5  122273643  122273767   chr5_122273693_122273717           chr5        122273684      122273700   

     align_strand  align_dist_to_signal align_N0_base  align_total_match  align_total_mismatch  \
0               -                     5             A                  7                     5   
1               +                    -2             A                 10                     2   
2               +                     5             A                  9                     3   
3               +                   -28             T                  7                     5   
4               -                    -2             T                  7                     5   
...           ...                   ...           ...                ...                   ...   
8520            -                   -63             C                 10                     7   
8521            +                   -23             T                  8                     9   
8522            +                    24             C                  9                     8   
8523            +                   -43             C                  9                     8   
8524            -                   -33             C                  8                     9   

      align_degen_total_match  align_degen_total_mismatch  align_degen_num  align_total_gap  align_score  \
0                          12                           0                5                0         50.0   
1                          11                           1                1                0         49.0   
2                          11                           1                2                0         47.0   
3                          10                           2                3                0         46.0   
4                          10                           2                3                0         46.0   
...                       ...                         ...              ...              ...          ...   
8520                       10                           7                0                0         22.0   
8521                        9                           8                1                0         21.0   
8522                       10                           7                1                0         20.0   
8523                       10                           7                1                0         20.0   
8524                       10                           7                2                0         18.0   

       align_target_seq   align_info_state    align_query_seq                                file_info  \
0          ATAAACTTTAAT       ||...||||..|       ATGGGCTTTGGT  ../art/DetectSeq_ATP8-DddA11_R

In [38]:
def get_fasta(x):
    region_index = x['region_index']
    seq = x['sequence']
    return f'>region_index:{region_index}\n{seq}\n'


for sample, df in df_arts_unassorted_add_seq.groupby('<sample>'):
    # 上面的一个cell中的代码已经证明了duplication是TAS-Dep_stat alignment过程引入的，简单去重即可
    df = df.drop_duplicates(subset=['region_index'], keep='first')
    text = ''.join(df.apply(get_fasta, axis=1).tolist())
    with open(f'../art_seqinfo/unassorted/seqinfo_{sample}_unassorted.fa', 'wt') as f:
        f.write(text)
# wc -l *.fa
#      218 seqinfo_DetectSeq_ATP8-DddA11_REP-1_unassorted.fa
#     1328 seqinfo_DetectSeq_ATP8-DddA6_REP-1_unassorted.fa
#       30 seqinfo_DetectSeq_ATP8-DddAwt_REP-1_unassorted.fa
#      638 seqinfo_DetectSeq_JAK2-DddA11_REP-1_unassorted.fa
#      946 seqinfo_DetectSeq_JAK2-DddA11_REP-2_unassorted.fa
#     4826 seqinfo_DetectSeq_SIRT6-DddA11_REP-1_unassorted.fa
#    17154 seqinfo_DetectSeq_SIRT6-DddA11_REP-2_unassorted.fa
#    25140 total


# total=25140/2=12570

In [39]:
df_arts_unassorted_add_seq

chrom      start        end               region_index align_chr_name  align_chr_start  align_chr_end  \
0      chr7  123412912  123413069   chr7_123412962_123413019           chr7        123412967      123412978   
1     chr19   45742345   45742506    chr19_45742395_45742456          chr19         45742447       45742458   
2     chr11  113102877  113103036  chr11_113102927_113102986          chr11        113102970      113102981   
3      chr4  127523422  127523578   chr4_127523472_127523528           chr4        127523489      127523500   
4     chr22   19547402   19547545    chr22_19547452_19547495          chr22         19547450       19547461   
...     ...        ...        ...                        ...            ...              ...            ...   
8520  chr10  128384449  128384564  chr10_128384499_128384514          chr10        128384451      128384467   
8521  chr12   15347569   15347765    chr12_15347619_15347715          chr12         15347722       15347738   
8522   chr8   10421053   10421176     chr8_10421103_10421126           chr8         10421063       10421079   
8523   chr1  165880712  165880857   chr1_165880762_165880807           chr1        165880789      165880805   
8524   chr5  122273643  122273767   chr5_122273693_122273717           chr5        122273684      122273700   

     align_strand  align_dist_to_signal align_N0_base  align_total_match  align_total_mismatch  \
0               -                     5             A                  7                     5   
1               +                    -2             A                 10                     2   
2               +                     5             A                  9                     3   
3               +                   -28             T                  7                     5   
4               -                    -2             T                  7                     5   
...           ...                   ...           ...                ...                   ...   
8520            -                   -63             C                 10                     7   
8521            +                   -23             T                  8                     9   
8522            +                    24             C                  9                     8   
8523            +                   -43             C                  9                     8   
8524            -                   -33             C                  8                     9   

      align_degen_total_match  align_degen_total_mismatch  align_degen_num  align_total_gap  align_score  \
0                          12                           0                5                0         50.0   
1                          11                           1                1                0         49.0   
2                          11                           1                2                0         47.0   
3                          10                           2                3                0         46.0   
4                          10                           2                3                0         46.0   
...                       ...                         ...              ...              ...          ...   
8520                       10                           7                0                0         22.0   
8521                        9                           8                1                0         21.0   
8522                       10                           7                1                0         20.0   
8523                       10                           7                1                0         20.0   
8524                       10                           7                2                0         18.0   

       align_target_seq   align_info_state    align_query_seq                                file_info  \
0          ATAAACTTTAAT       ||...||||..|       ATGGGCTTTGGT  ../art/DetectSeq_ATP8-DddA11_R

# Motif analysis
```bash
see file meme_chip_step_by_step.sh
mkdir -p \
    motif_pre_processing \
    motif_discovery/meme \
    motif_discovery/streme \
    motif_scanning/fimo \
    motif_comparison/tomtom \
    motif_enrichment/centrimo \
    motif_enrichment/spamo


for i in `ls *.fa`
# for i in seqinfo_DetectSeq_ATP8-DddA11_REP-1_independent.fa  # for test
do
    ########################################
    #### files ####
    ########################################
    # motif_pre_processing
    SEQ_LENGTH=400
    METRICS=motif_pre_processing/${i}.metrics
    SEQ_CENTERED=motif_pre_processing/${i}.seq_centered
    BFILE=motif_pre_processing/${i}.background
    
    ### Motif Discovery
    ### 预测输入序列上的motif信息
    ### de novo motif discovery
    MEME=motif_discovery/meme/$i
    STREME=motif_discovery/streme/$i

    ### Motif Enrichment
    ### 分析已知的motif在输入序列上的富集情况
    CENTRIMO=motif_enrichment/centrimo/$i
    SPAMO=motif_enrichment/spamo/$i

    # ### Motif Scanning
    # ### 分析输入序列上可能的motif出现的位置
    FIMO=motif_scanning/fimo/$i
    
    # ### Motif Comparison
    # ### 比较不同motif之间的相似性
    TOMTOM=motif_comparison/tomtom/${i}
    

    
    echo '===================================================>'
    echo 'pre-processing' # √
    echo '===================================================>'
    getsize $i 1> $METRICS
    fasta-most -min 50 < $i 1>> $METRICS  # test
    fasta-center -dna -len $SEQ_LENGTH < $i 1> $SEQ_CENTERED
    fasta-get-markov -nostatus -nosummary -dna -m 2 $i $BFILE

 
    echo '===================================================>'
    echo 'Motif Discovery: de novo motif discovery' # √
    echo '===================================================>'
    echo -e '\trun MEME'
    meme -dna -nmotifs 3 -maxw 50 -bfile $BFILE -revcomp -oc $MEME $i
    # nmotifs: 最多报告3个motifs
    # revcomp: allow sites on + or - DNA strands
    # nostatus: do not print progress reports to terminal
    
    echo -e '\trun STREME'
    streme -dna --nmotifs 3 --maxw 30 --p $SEQ_CENTERED --oc $STREME
    # STREME: The maximum allowed motif width is 30


    echo '===================================================>'
    echo 'Motif Enrichment: 分析已知的motif在输入序列上的富集情况' # √
    echo '===================================================>'
    echo -e '\trun CentiMo'
    centrimo -seqlen $SEQ_LENGTH -bfile $BFILE $SEQ_CENTERED $MEME/meme.xml \
     $STREME/streme.xml \
     -oc $CENTRIMO
 
    echo -e '\trun SpaMo'
    spamo -bgfile $BFILE -keepprimary -oc $SPAMO $i $MEME/meme.xml $STREME/streme.xml

    
    echo '===================================================>'
    echo 'motif scanning'  # √
    echo '===================================================>'
    echo -e '\trun FIMO'
    fimo --oc $FIMO --bgfile $BFILE $MEME/meme.xml $i

    echo '===================================================>'
    echo 'motif comparison'  # √
    echo '===================================================>'

    echo -e '\trun TOMTOM'
    tomtom -oc $TOMTOM $MEME/meme.xml \
     $HOME/1.database/db_meme_suite/motif_databases/JASPAR/JASPAR2022_CORE_redundant_v2.meme


done
```

## run scripts above, wait and next
在veteran服务器上跑，剩下的继续在本地分析
多拆几个脚本出来

# 三种类型脱靶的统计

In [40]:
import pandas as pd

In [43]:
ls = []
# df = pd.read_csv('../art_seqinfo/df_arts_tas_dep.csv')
df = pd.read_csv('../art_seqinfo/df_arts_tas_dep_v2.csv')
ls.append(df)
print(df['TAS-Dep_stat'].value_counts())
# df = pd.read_csv('../art_seqinfo/df_arts_tas_indep_add_seq.csv')
df = pd.read_csv('../art_seqinfo/df_arts_tas_indep_add_seq_v3.csv')
df.loc[~df['TAS-Dep_stat'].str.contains('Motif'), 'TAS-Dep_stat'] = 'CTCF'
df.loc[df['TAS-Dep_stat'].str.contains('Motif'), 'TAS-Dep_stat'] = 'CTCF_additional'
ls.append(df)
print(df['TAS-Dep_stat'].value_counts())
# df = pd.read_csv('../art_seqinfo/df_arts_unassorted_add_seq.csv')
df = pd.read_csv('../art_seqinfo/df_arts_unassorted_add_seq_v3.csv')
df['TAS-Dep_stat'] = 'Unassorted'
ls.append(df)

df = pd.concat(ls)
df

TAS-Dep_stat
Left_additional     1328
Right                830
Right_additional     302
Left                 260
Both_additional       67
Both                  41
Name: count, dtype: int64
TAS-Dep_stat
CTCF_additional    2348
CTCF               1838
Name: count, dtype: int64


chrom      start        end               region_index align_chr_name  align_chr_start  align_chr_end  \
0     chr10  123171455  123171602  chr10_123171505_123171552          chr10        123171536      123171547   
1      chr1  170722730  170722881   chr1_170722780_170722831           chr1        170722814      170722825   
2      chr3   37937199   37937325     chr3_37937249_37937275           chr3         37937253       37937264   
3      chr9  109215639  109215812   chr9_109215689_109215762           chr9        109215772      109215783   
4      chr4    6913404    6913549       chr4_6913454_6913499           chr4          6913483        6913494   
...     ...        ...        ...                        ...            ...              ...            ...   
8520  chr10  128384449  128384564  chr10_128384499_128384514          chr10        128384451      128384467   
8521  chr12   15347569   15347765    chr12_15347619_15347715          chr12         15347722       15347738   
8522   chr8   10421053   10421176     chr8_10421103_10421126           chr8         10421063       10421079   
8523   chr1  165880712  165880857   chr1_165880762_165880807           chr1        165880789      165880805   
8524   chr5  122273643  122273767   chr5_122273693_122273717           chr5        122273684      122273700   

     align_strand  align_dist_to_signal align_N0_base  align_total_match  align_total_mismatch  \
0               +                     5             A                 12                     0   
1               +                     6             A                 11                     1   
2               -                     4             A                 11                     1   
3               -                    10             A                 11                     1   
4               +                     5             A                 10                     2   
...           ...                   ...           ...                ...                   ...   
8520            -                   -63             C                 10                     7   
8521            +                   -23             T                  8                     9   
8522            +                    24             C                  9                     8   
8523            +                   -43             C                  9                     8   
8524            -                   -33             C                  8                     9   

      align_degen_total_match  align_degen_total_mismatch  align_degen_num  align_total_gap  align_score  \
0                          12                           0                0                0         60.0   
1                          12                           0                1                0         58.0   
2                          12                           0                1                0         58.0   
3                          12                           0                1                0         58.0   
4                          12                           0                2                0         56.0   
...                       ...                         ...              ...              ...          ...   
8520                       10                           7                0                0         22.0   
8521                        9                           8                1                0         21.0   
8522                       10                           7                1                0         20.0   
8523                       10                           7                1                0         20.0   
8524                       10                           7                2                0         18.0   

       align_target_seq   align_info_state    align_query_seq                                file_info  \
0          ATGGGCTTTGGT       ||||||||||||       ATGGGCTTTGGT  ../art/DetectSeq_ATP8-DddA11_R

In [44]:
print(df['TAS-Dep_stat'].value_counts())

TAS-Dep_stat
Unassorted          8525
CTCF_additional     2348
CTCF                1838
Left_additional     1328
Right                830
Right_additional     302
Left                 260
Both_additional       67
Both                  41
Name: count, dtype: int64


In [45]:
# cell: from step 12 , save all reported list
df.drop(columns=['id', 'ctcf_chrom', 'ctcf_start', 'ctcf_end', 'sequence'], inplace=True)

In [46]:
# for step 08 plot circos plot
df.to_csv('../final_list_after_igv_check/2023-08-25_merged_final_list_after_classification_step11.csv', index=False)

In [50]:
df_counts = df.groupby(['<sample>', 'TAS-Dep_stat'])['region_index'].count().to_frame().reset_index().pivot(columns='TAS-Dep_stat', index='<sample>', values='region_index').fillna(0).astype(int)
df_counts['total'] = df_counts.sum(axis=1)
df_counts = df_counts[['Left', 'Left_additional', 'Right', 'Right_additional', 'Both', 'Both_additional', 'CTCF', 'CTCF_additional', 'Unassorted', 'total']]
df_counts.rename(columns={'total': 'Total'}, inplace=True)
df_counts

TAS-Dep_stat                  Left  Left_additional  Right  Right_additional  Both  Both_additional  CTCF  \
<sample>                                                                                                    
DetectSeq_ATP8-DddA11_REP-1      1                0     99                 0     6                0    52   
DetectSeq_ATP8-DddA6_REP-1      14                0    454                 0    20                0    34   
DetectSeq_ATP8-DddAwt_REP-1      1                0      7                 0     0                0     6   
DetectSeq_JAK2-DddA11_REP-1     22               10     11                 5     1                1   428   
DetectSeq_JAK2-DddA11_REP-2     15               11    162                50     9                8   183   
DetectSeq_SIRT6-DddA11_REP-1    44              312     27                42     0               15   611   
DetectSeq_SIRT6-DddA11_REP-2   163              995     70               205     5               43   524   

TAS-Dep_stat                  CTCF_additional  Unassorted  Total  
<sample>                                                          
DetectSeq_ATP8-DddA11_REP-1                11          98    267  
DetectSeq_ATP8-DddA6_REP-1                  5         659   1186  
DetectSeq_ATP8-DddAwt_REP-1                 2          13     29  
DetectSeq_JAK2-DddA11_REP-1               217          86    781  
DetectSeq_JAK2-DddA11_REP-2               136         268    842  
DetectSeq_SIRT6-DddA11_REP-1              565        1479   3095  
DetectSeq_SIRT6-DddA11_REP-2             1412        5922   9339

In [51]:
print(df_counts.to_markdown())

| <sample>                     |   Left |   Left_additional |   Right |   Right_additional |   Both |   Both_additional |   CTCF |   CTCF_additional |   Unassorted |   Total |
|:-----------------------------|-------:|------------------:|--------:|-------------------:|-------:|------------------:|-------:|------------------:|-------------:|--------:|
| DetectSeq_ATP8-DddA11_REP-1  |      1 |                 0 |      99 |                  0 |      6 |                 0 |     52 |                11 |           98 |     267 |
| DetectSeq_ATP8-DddA6_REP-1   |     14 |                 0 |     454 |                  0 |     20 |                 0 |     34 |                 5 |          659 |    1186 |
| DetectSeq_ATP8-DddAwt_REP-1  |      1 |                 0 |       7 |                  0 |      0 |                 0 |      6 |                 2 |           13 |      29 |
| DetectSeq_JAK2-DddA11_REP-1  |     22 |                10 |      11 |                  5 |      1 |                 1 